In [57]:
import disp2depth
import cv2
import numpy as np
import re

def read_pfm(filename):
    '''Source: MVSNet/mvsnet/preprocess.py'''
    
    file = open(filename, 'rb')
    color = None
    width = None
    height = None
    scale = None
    data_type = None
    header = file.readline().decode('UTF-8').rstrip()

    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')
    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode('UTF-8'))
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')
    # scale = float(file.readline().rstrip())
    scale = float((file.readline()).decode('UTF-8').rstrip())
    if scale < 0: # little-endian
        data_type = '<f'
    else:
        data_type = '>f' # big-endian
    data_string = file.read()
    data = np.frombuffer(data_string, data_type)
    shape = (height, width, 3) if color else (height, width)
    data = np.reshape(data, shape)
    data = cv2.flip(data, 0)
    return data


def read_disparity_map(path: str) -> np.ndarray:
    """
    Basé sur le fichier disparity_reader.py du dataset Revery
    Comme les fichiers .fpm prennent beaucoup de places,
    il a été utilisé des fichiers .png pour prendre moins de place dans lesquels
    la disparité est packée suivant la formule dans le code
    
    Returns:
        La carte de disparité dans un tableau 2D
    """
    
    tmp = cv2.imread(path, -1)

    try:
        # print(path)
        b, g, r, a = cv2.split(tmp)

    except:
        print(path)
        exit(1)

    res = r + g / 256.0 + b / (256.0 * 256.0) + a / (256.0 * 256.0 * 256.0)

    return 32 * res

id = "aa6r0lw3lkqv9m108bvpxm"
cam = 9
disp_path = f"/scratch_p/rafrantz/.revery-original/{id}depth{cam}.png"
depth_path = f"/scratch_p/rafrantz/revery/{id}/rendered_depth_maps/0000000{cam}.pfm"

disp_im = read_disparity_map(disp_path)
depth_im = read_pfm(depth_path)

pixel = (0, 0)

print("disparity =", disp_im[pixel])
print("depth =", depth_im[pixel])


baseline = 0.30
image_height = 1080
fovy = np.radians(60)

disparity_to_depth_numerator = (
    (baseline * image_height) /
    (2 * np.tan(fovy / 2))
)

for m in ["min", "max"]:
    print(f"{m}_depth = ", getattr(np, m)(depth_im))
    print(f"{m}_disp = ", getattr(np, m)(disp_im))
    
print("conversion: ", disparity_to_depth_numerator / 280)

disparity = 196.03954887390137
depth = 1.4313042
min_depth =  1.409572
min_disp =  5.126462936401367
max_depth =  54.73408
max_disp =  199.06200981140137
conversion:  1.002115110093422
